In [1]:
import requests
import json
import spotipy
import pandas as pd
from typing import List, Dict
import time

In [2]:
import spotipy.util as util

# Lấy token để gọi API

In [10]:
username = "Chrisnguyen"
client_id= "ac6f1d431d774310a0761aef45b2160a"
client_secret = "c4142f832771476ea77e178bff1c9b3d"
redirect_uri  = "http://localhost:8080/callback/"
scope = 'playlist-read-collaborative'

def renew_token():
    return util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

token = renew_token()

In [11]:
print(token)

BQDjO4oWYsjN7StEQIjOraQ6S3WS6o9dzhGqRe5cx5BuZQYNUAn25jwbORpxq-fpZ7d0ExVq39zy-HG24hztz9JV0Lbt17AOLvtkftvlmWNCXMGvQus6VZIyR-wP5QiKBfWdfLi32URvKvxHz0__7MKKhzH8VSJqWNipRfCijAHbcGW7NPtv58iRcmmjmeLJdy1FRgI


In [5]:
def get_id(track_name: str, token: str) -> str:
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': f'Bearer ' + token,
    }
    params = [
        ('q', track_name),
        ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None

In [6]:
def get_features(track_id: str, token: str) -> dict:
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }

    response = requests.get(f"https://api.spotify.com/v1/audio-features/{track_id}", headers = headers)
    json_obj = response.json()
    
    if response.status_code != 200:
        print(response)
        return response, None
    
    return response, json_obj

# Lấy danh sách các playlist do spotify tổng hợp

In [7]:
def get_all_playlist(username: str, token: str) -> List[Dict]:
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }
    
    offset   = 0
    limit    = 50
    total    = 100000 #max offset in spotify
    playlist = []
    
    while offset < total:
        params = [("limit", limit), ("offset", offset)]
        
        response = requests.get(f"https://api.spotify.com/v1/users/{username}/playlists", headers = headers, params=params)
        json_obj = response.json()
        
        if response.status_code != 200:
            print(response)
            continue
        
        playlist.extend(json_obj["items"])
        n = offset + len(json_obj["items"])

        print(f"Successfully get from playlist {offset} to {n}")
        offset = n
        
        if json_obj["total"] < total:
            total = json_obj["total"]
            
        if not "next" in json_obj or json_obj["next"] == "":
            break

    return playlist

In [117]:
spotify_playlist = get_all_playlist("spotify", token)

Successfully get from playlist 0 to 50
Successfully get from playlist 50 to 100
Successfully get from playlist 100 to 150
Successfully get from playlist 150 to 200
Successfully get from playlist 200 to 250
Successfully get from playlist 250 to 300
Successfully get from playlist 300 to 350
Successfully get from playlist 350 to 400
Successfully get from playlist 400 to 450
Successfully get from playlist 450 to 500
Successfully get from playlist 500 to 550
Successfully get from playlist 550 to 600
Successfully get from playlist 600 to 650
Successfully get from playlist 650 to 700
Successfully get from playlist 700 to 750
Successfully get from playlist 750 to 800
Successfully get from playlist 800 to 850
Successfully get from playlist 850 to 900
Successfully get from playlist 900 to 950
Successfully get from playlist 950 to 1000
Successfully get from playlist 1000 to 1050
Successfully get from playlist 1050 to 1100
Successfully get from playlist 1100 to 1150
Successfully get from playlist 

In [118]:
playlist_id = [s["id"] for s in spotify_playlist]    

In [119]:
with open("playlist.txt", "w") as f:
    for ids in playlist_id:
        f.write(ids + "\n")

In [121]:
len(playlist_id)

1412

# Lấy tất cả các dữ liệu về track

In [7]:
def get_row_from_trackid(ids: str, token: str):
    #Lấy thông tin về track
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }

    response = requests.get(f"https://api.spotify.com/v1/tracks/{ids}", headers = headers)
    json_obj = response.json()
    
    if response.status_code != 200:
        print(response)
        return response, None
    #Lấy đặc trưng của track
    response, fts  = get_features(ids, token)
    if response.status_code != 200:
        print(response)
        return response, None
    
    #Tạo dòng
    row = {}
    row["artists"] = ",".join([a["id"] for a in json_obj["artists"]])
    row["available_markets"] = ",".join(json_obj["available_markets"])
    row["explicit"]   = json_obj["explicit"]
    row["popularity"] = json_obj["popularity"]
    row.update(fts)
    return response, row

In [22]:
get_row_from_trackid("2QDV31b1RcaMUxKZJ0hJld", token)

(<Response [200]>,
 {'artists': '3NBrt8JfhJ7ucWdqrsxQbU',
  'available_markets': 'AD,AE,AL,AR,AT,AU,BA,BE,BG,BH,BO,BR,BY,CA,CH,CL,CO,CR,CY,CZ,DE,DK,DO,DZ,EC,EE,EG,ES,FI,FR,GB,GR,GT,HK,HN,HR,HU,ID,IE,IL,IN,IS,IT,JO,JP,KW,KZ,LB,LI,LT,LU,LV,MA,MC,MD,MK,MT,MX,MY,NI,NL,NO,NZ,OM,PA,PE,PH,PL,PS,PT,PY,QA,RO,RU,SA,SE,SG,SI,SK,SV,TH,TN,TR,TW,UA,US,UY,VN,ZA',
  'explicit': False,
  'popularity': 10,
  'danceability': 0.696,
  'energy': 0.45,
  'key': 8,
  'loudness': -13.074,
  'mode': 0,
  'speechiness': 0.953,
  'acousticness': 0.821,
  'instrumentalness': 0,
  'liveness': 0.0973,
  'valence': 0.391,
  'tempo': 103.892,
  'type': 'audio_features',
  'id': '2QDV31b1RcaMUxKZJ0hJld',
  'uri': 'spotify:track:2QDV31b1RcaMUxKZJ0hJld',
  'track_href': 'https://api.spotify.com/v1/tracks/2QDV31b1RcaMUxKZJ0hJld',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2QDV31b1RcaMUxKZJ0hJld',
  'duration_ms': 1104427,
  'time_signature': 4})

In [47]:
def get_all_track(token: str, start: int = 0, max_num: int = 10000) -> List[Dict]:
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }
    tracks = []
    
    num_track = start
    consecutive_fails = 0
    with open("tracks.txt") as f:
        # skip tới id thử start
        for i in range(start):
            next(f)
                
        for line in f:
            try:
                response, row = get_row_from_trackid(line[:-1], token)
                
                if response.status_code == 200:
                    # Nhận thành công
                    consecutive_fails = 0
                    tracks.append(row)
                    num_track += 1
                    if num_track % 1000 == 0:
                        print("Get 1000 track")
                elif response.status_code == 401:
                    print(response)
                    # Hết hạn token
                    consecutive_fails += 1
                    token = renew_token()
                elif response.status_code == 404:
                    print(response)
                    # Không tìm thầy đường dẫn
                    consecutive_fails += 1
                    pass
            except:
                consecutive_fails += 1
                pass
            
            if consecutive_fails == 10 or num_track >= max_num:
                A = pd.DataFrame(tracks)
                A.to_csv(f"track_{start}_{num_track}.csv")
                return num_track
                
            if num_track % 1000 == 0:
                print("Get 500 track")

In [48]:
num_track = get_all_track(token)

<Response [401]>
<Response [401]>
<Response [401]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
Get 1000 track
Get 500 track
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
Get 1000 track
Get 500 track
Get 1000 track
Get 500 track
Get 1000 track
Get 500 track
<Response [404]>
<Response [404]>
Get 1000 track
Get 500 track
<Response [404]>
<Response [404]>
Get 1000 track
Get 500 track
<Response [401]>
<Response [401]>
<Response [401]>
Get 1000 track
Get 500 track
Get 1000 track
Get 500 track
Get 1000 track
Get 500 track
Get 1000 track


In [49]:
num_track

10000

# Lấy tất cả các dữ liệu về artist

In [39]:
ids = "6jMfbm9y64CESMB5wFtgZx"

In [50]:
def get_row_from_artistid(ids: str, token: str)-> Dict[str, object]:
    #Lấy thông tin về track
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }

    response = requests.get(f"https://api.spotify.com/v1/artists/{ids}", headers = headers)
    json_obj = response.json()

    if response.status_code != 200:
        return response, None
    #Tạo dòng
    row = {}
    row["followers"] = json_obj["followers"]["total"]
    row['genres'] = json_obj['genres']
    row['id'] = json_obj['id']
    row['name'] = json_obj['name']
    row['popularity'] = json_obj['popularity']

    return response, row

In [47]:
get_row_from_artistid(ids, token)

{'followers': 15975,
 'genres': ['swedish reggae'],
 'id': '6jMfbm9y64CESMB5wFtgZx',
 'name': 'Governor Andy',
 'popularity': 38}

In [55]:
def get_all_artist(token: str, start: int = 0, max_num: int = 10000) -> List[Dict]:
    headers = {
        'Accept': 'application/json',\
        'Content-Type': 'application/json',\
        'Authorization': f'Bearer ' + token,\
    }
    artists = []
    
    num_artist = start
    consecutive_fails = 0
    with open("artists.txt") as f:
        # skip tới id thử start
        for i in range(start):
            next(f)
                
        for line in f:
            try:
                response, row = get_row_from_artistid(line[:-1], token)
                
                if response.status_code == 200:
                    # Nhận thành công
                    consecutive_fails = 0
                    artists.append(row)
                    num_artist += 1
                    if num_artist % 1000 == 0:
                        print("Get 1000 artists")
                elif response.status_code == 401:
                    # Hết hạn token
                    consecutive_fails += 1
                    token = renew_token()
                elif response.status_code == 404:
                    print(response)
                    # Không tìm thầy đường dẫn
                    consecutive_fails += 1
                    pass
            except:
                consecutive_fails += 1
                pass
            
            if consecutive_fails == 10 or num_artist >= max_num:
                A = pd.DataFrame(artists)
                A.to_csv(f"artist_{start}_{num_artist}.csv")
                return num_artist
                
        A = pd.DataFrame(artists)
        A.to_csv(f"artist_{start}_{num_artist}.csv")
        return num_artist

In [56]:
num_artist = get_all_artist(token)

Get 1000 artists
Get 1000 artists
Get 1000 artists
Get 1000 artists
<Response [404]>
<Response [404]>
Get 1000 artists
Get 1000 artists
Get 1000 artists
Get 1000 artists
Get 1000 artists
Get 1000 artists


In [57]:
num_artist

10000